# ML-Ask を用いたツイートの感情解析

## 感情解析ライブラリ ML-Ask
ML-Ask とは中村[1]の提唱した 10種類の感情モデルを基に，感情表現辞典に記載された表現をキーワードとして探索し，感情を推定する手法[2, 3]である。

感情は喜, 怒, 哀, 怖, 恥, 好, 厭, 昂, 安, 驚の10種類に分類される。

ML-Ask に関する概要は以下の Web ページに記載されている。
> http://arakilab.media.eng.hokudai.ac.jp/~ptaszynski/repository/mlask.htm

[1] 中村明, 感情表現辞典, 東京堂出版, 1993.

[2] Michal Ptaszynski, Pawel Dybala, Rafal Rzepka and Kenji Araki, “Affecting Corpora: Experiments with Automatic Affect Annotation System - A Case Study of the 2channel Forum -”, In Proceedings of The Conference of the Pacific Association for Computational Linguistics (PACLING-09), September 1-4, 2009, Hokkaido University, Sapporo, Japan, pp. 223-228.

[3] Michal Ptaszynski, Pawel Dybala, Wenhan Shi, Rafal Rzepka and Kenji Araki, “A System for Affect Analysis of Utterances in Japanese Supported with Web Mining”, Journal of Japan Society for Fuzzy Theory and Intelligent Informatics, Vol. 21, No. 2 (April), pp. 30-49 (194-213), 2009.

### pymlask のインストール
ML-Ask を Python で実装したライブラリが pymlask である。

https://github.com/ikegami-yukino/pymlask?files=1

Anaconda3 から Anaconda Prompt を起動し，以下の pip コマンドを入力すると pymlask がインストールされる。（ここで先頭の "$" はプロンプトである)

`$ pip install pymlask`

なお，ML-Ask の動作には MeCab が必要となる。（前回の実験でインストール済みのはず）

### ML-Ask による感情推定(1)
以下は感情をひとつだけ含む例である。「美味しくない」に対して「厭」の感情が紐付けられている。

また，「美味しい」＋「ない」から negation も考慮されていることが分かる(ML-Ask では Contextual Valence Shifters (CVS)により negation および intensifier を判定する)。

(参考) ML-Ask では感情の他に極性(orinetation)，態(activation; 能動的な受動的か)も判定する。
以下の例では極性が "NEGATIVE", 態が "NEUTRAL" と判定されている。

In [8]:
from mlask import MLAsk
emotion_analyzer = MLAsk()
result = emotion_analyzer.analyze("バナナは美味しくない")
result

{'text': 'バナナは美味しくない',
 'emotion': defaultdict(list, {'iya': ['美味しい*CVS']}),
 'orientation': 'NEGATIVE',
 'activation': 'NEUTRAL',
 'emoticon': None,
 'intension': 0,
 'intensifier': {},
 'representative': ('iya', ['美味しい*CVS'])}

### ML-Ask による感情推定(2)
以下はひとつのテキストに複数の感情を含む例である。
ここでは「心配」，「楽しみ」がそれぞれ「怖」「喜」に対応する感情語として抽出されている。
また，代表的な感情(representative)として「喜」が推定される。

なお，極性は "NEUTRAL"，態は "ACTIVE" である。

In [18]:
text = "コロナウイルスの広がりが心配だ。でも，明日のボーナスが楽しみ！"
result = emotion_analyzer.analyze(text)
result

{'text': 'コロナウイルスの広がりが心配だ。でも，明日のボーナスが楽しみ！',
 'emotion': defaultdict(list, {'kowa': ['心配'], 'yorokobi': ['楽しみ']}),
 'orientation': 'NEUTRAL',
 'activation': 'ACTIVE',
 'emoticon': None,
 'intension': 2,
 'intensifier': {'interjections': ['ウイ'], 'exclamation': ['！']},
 'representative': ('yorokobi', ['楽しみ'])}

## ML-Ask を用いた COVID-19 関連ツイートの分析

### 適当なテキストを与えて推定される感情を調べてみる
以下では 2020年4月1日の COVID-19 関連ツイート群を与え，それぞれのツイート（テキスト）の感情を推定する。
結果から明らかなように，感情を含まないツイートも存在する。

In [21]:
def emotion_estimation(filename, max_count=10):
    emotion_analyzer = MLAsk()
    
    with open(filename, "r", encoding='utf-8') as f:
        for line in f.readlines()[:max_count]:
            # 何故か分からないが，(id, retweet_count 等が含まれないテキストのみの行が存在する
            try:
                status_id, retweet_count, favorite_count, text = line.rstrip('\n').split('\t')
            except ValueError:
                continue
                
            status_id = int(status_id)
            retweet_count = int(retweet_count)
            favorite_count = int(favorite_count)
            
            # URL を含むツイートを排除（超簡易版の実装）
            if "http" in text:
                continue
                
            result = emotion_analyzer.analyze(text)
            print("--------------------")
            print(text)
            print(result)

emotion_estimation("/home/muto/Dropbox/ExpSSE3/Tweets/20200401_MERS.txt", max_count=20)


--------------------
@pe_pe_pe_pen @nhk_news 新型コロナウイルスだったみたいですね
{'text': '@pe_pe_pe_pen @nhk_news 新型コロナウイルスだったみたいですね', 'emotion': None}
--------------------
【在バンクーバー日本総領事館からのお知らせ②】1/2  ●４月１日、日本政府は、新型コロナウイルス感染症の拡大に伴う水際対策強化に係る新たな措置として、カナダを含む４９か国・地域からの日本人を含む入国者に対し、PCR検査の実施、検疫所長の指定する場所での１４日間の待機、
{'text': '【在バンクーバー日本総領事館からのお知らせ②】1/2  ●４月１日、日本政府は、新型コロナウイルス感染症の拡大に伴う水際対策強化に係る新たな措置として、カナダを含む４９か国・地域からの日本人を含む入国者に対し、PCR検査の実施、検疫所長の指定する場所での１４日間の待機、', 'emotion': None}
--------------------
気づけば、アメリカでのコロナウイルスによる死亡者は、同時多発テロでの死亡者を軽く超えてしまってる‥  さぁ、今日あなたは感染拡大させない為に何が出来ますか？ 何をして人の命を守りますか？  私は今日搭乗予定だった飛行機をキャンセルしました。  #感染予防のためにできること
{'text': '気づけば、アメリカでのコロナウイルスによる死亡者は、同時多発テロでの死亡者を軽く超えてしまってる‥  さぁ、今日あなたは感染拡大させない為に何が出来ますか？ 何をして人の命を守りますか？  私は今日搭乗予定だった飛行機をキャンセルしました。  #感染予防のためにできること', 'emotion': None}
--------------------
まじ、満員電車やだ、 隠れミッキーのように潜んでいるんだろうな、コロナウイルスの試薬さえあれば、
{'text': 'まじ、満員電車やだ、 隠れミッキーのように潜んでいるんだろうな、コロナウイルスの試薬さえあれば、', 'emotion': None}
--------------------
ワイドショー見てる  コロナでTV業界も打撃とか言い出したからTBSに呆れた  読売TVは京大准教

### 一定期間内の感情の推移
連日，多数のツイートが投稿される。

以下では 2020年3月1日〜2020年3月31日に投稿されたツイートに含まれる感情の分布を分析する。
この期間は，日本において小中高校の臨時休校が始まり，新型コロナウイルス感染者数が増加し続け，「都市部において緊急事態宣言が出されるか？」と国民が疑心暗鬼に陥っていた時期である。

#### (可視化の準備) Plotly Express のインストール
Anaconda Prompt にて以下の pip コマンドを入力すること

`$ pip install plotly_express`

Plotly Express の詳細は下記のサイトを参照のこと：
- https://plotly.com/python/plotly-express/
- https://qiita.com/hanon/items/d8cbe25aa8f3a9347b0b

#### プログラムの全体像
プログラムは

- 感情を推定する emotion_estimation()
- 分析期間を設定する set_period()
- 全体の制御 ＆ 可視化を行うメイン関数

から構成される。

In [26]:
# ML-Ask の出力する結果のうち representative を用いた結果
from mlask import MLAsk
import datetime
import sys

# 感情推定
def emotion_estimation(filename, emotions, verb=False):
    import os
    if os.path.exists(filename) is False:
        return []
    
    # ML-Ask コンストラクタ
    emotion_analyzer = MLAsk()

    # 感情の数を収める辞書の初期化
    emotion_counter = dict()
    for emotion in emotions:
        emotion_counter[emotion] = 0

    # 感情を含むツイートの総数をカウント
    emotional_tweet_counter = 0
    
    '''
    ファイルの形式は
    status_id \t retweet_count \t favorite_coun \t text
    '''
    with open(filename, "r", encoding='utf-8') as f:
        for line in f.readlines():
            # 何故か分からないが，(id, retweet_count 等が含まれないテキストのみの行が存在する
            try:
                status_id, retweet_count, favorite_count, text = line.rstrip('\n').split('\t')
            except ValueError:
                continue
                
            status_id = int(status_id)
            retweet_count = int(retweet_count)
            favorite_count = int(favorite_count)
            
            # URL を含むツイートを排除（超簡易版の実装）
            if "http" in text:
                continue

            # 感情解析
            result = emotion_analyzer.analyze(text)
            if 'representative' in result:
                emotional_tweet_counter += 1 # 感情を含むツイート数
                emotion = result['representative'][0] # 代表的な感情のみ取り出す
                emotion_counter[emotion] += 1
                
                # デバッグ用
                if verb == True:
                    print("--------------------")
                    print(text)
                    print(emotion)

    # 感情カウント値の正規化
    for key in emotion_counter:
        emotion_counter[key] /= emotional_tweet_counter
        
    return emotion_counter

'''
データを収集する期間を指定
dates = ["20200401", "20200402", ...., "20200630"] というリストを作る
'''
def set_period(from_year, from_month, from_day, until_year, until_month, until_day):
    dates = list()

    d = datetime.datetime(from_year, from_month, from_day)
    while(True):
        if d > datetime.datetime(until_year, until_month, until_day): break
        dates.append("{}{:02d}{:02d}".format(d.year, d.month, d.day))
        d += datetime.timedelta(days=1)
    
    return dates

if __name__ == '__main__':
    # 開始時刻をメモ
    start_time = datetime.datetime.now()
    
    results = dict()

    # ML-Ask にて定義される感情の一覧
    # 可視化の際，日本語を用いるため ML-Ask の出力との対応表も準備する
    emotions = ['iya', 'odoroki', 'yasu', 'yorokobi', 'kowa', 'suki', 'takaburi', 'ikari', 'aware', 'haji']
    j_emotions = {'iya':'厭', 'odoroki':'驚', 'yasu':'安', 'yorokobi':'喜', 'kowa':'怖', 'suki':'好', 'takaburi':'昂', 'ikari':'怒', 'aware':'哀', 'haji':'恥'}

    # データ収集期間の指定
    dates = set_period(2020, 3, 1, 2020, 3, 31)

    # 各月日の感情を推定 ＆ カウントアップ
    for d in dates:
        # ファイル名，特にパスは各自の環境に合わせて！
        filename = "/home/muto/Dropbox/ExpSSE3/Tweets/{}_MERS.txt".format(d)        
        
        # 感情推定
        emotion_counter = emotion_estimation(filename, emotions)
        print(emotion_counter, file=sys.stderr)
        
        # データを取れていない日に対処するための措置
        if emotion_counter == []: continue

        # 月日 "d" の感情カウントを保存
        results[d] = emotion_counter
    
    # 終了時刻をメモ
    end_time = datetime.datetime.now()
    
    print("処理時間 {}".format(end_time - start_time), file=sys.stderr)
    
    # pandas データフレームに値を押し込む
    # 理由１：Plotly Express で処理するため
    # 理由２：感情を日本語で表現するため
    import pandas as pd
    df = pd.DataFrame(columns=['date']+emotions)
    
    #import matplotlib.pyplot as plt
    #from matplotlib.font_manager import FontProperties
    #fp = FontProperties(fname=r'/usr/share/fonts/truetype/fonts-japanese-gothic.ttf', size=16)

    for d in dates:
        dt = datetime.datetime(int(d[0:4]), int(d[4:6]), int(d[6:8]))
        try:
            df = df.append(pd.DataFrame([[dt, results[d]['iya'], results[d]['odoroki'], results[d]['yasu'],
                                          results[d]['yorokobi'], results[d]['kowa'],results[d]['suki'],
                                          results[d]['takaburi'], results[d]['ikari'], results[d]['aware'],
                                          results[d]['haji']]],
                                         columns=['date']+emotions))
        except:
            pass # データを取れていない日に対処するための措置
    
    # 列名をローマ字表記から日本語表記へリネーム
    df = df.rename(columns=j_emotions)
    
    df = df.reset_index()
    
    print(df)

    # Plotly Express を用いた可視化
    import plotly_express as px
    df_melt = df.melt(id_vars='date', value_vars=j_emotions.values()) # value_vars=emotions)
    fig = px.line(df_melt, x="date", y="value", color='variable',
                  title="Emotions ....")
    fig.show()

{'iya': 0.3366082086126159, 'odoroki': 0.026443852722431365, 'yasu': 0.09778716371315765, 'yorokobi': 0.1502157744927004, 'kowa': 0.2008998255440272, 'suki': 0.047837664126342855, 'takaburi': 0.011477366632999725, 'ikari': 0.0499494995868148, 'aware': 0.0754751629786062, 'haji': 0.0033054815903039206}
{'iya': 0.3642112763872807, 'odoroki': 0.025474303019506546, 'yasu': 0.0940589649951011, 'yorokobi': 0.14278079629464682, 'kowa': 0.17716219827202281, 'suki': 0.04506992072681928, 'takaburi': 0.014874855259641934, 'ikari': 0.060033846976039903, 'aware': 0.07268192749621448, 'haji': 0.003651910572726463}
{'iya': 0.35136295342348145, 'odoroki': 0.024254131787937325, 'yasu': 0.09626529298132647, 'yorokobi': 0.15647134578235672, 'kowa': 0.1837304142519854, 'suki': 0.04282034771410174, 'takaburi': 0.014273449238033913, 'ikari': 0.0540888602704443, 'aware': 0.0721184803605924, 'haji': 0.0046147241897402875}
{'iya': 0.3569723832253665, 'odoroki': 0.022388907830435276, 'yasu': 0.09910217070121605

{'iya': 0.35974945533769065, 'odoroki': 0.025326797385620915, 'yasu': 0.09531590413943355, 'yorokobi': 0.13711873638344227, 'kowa': 0.1880446623093682, 'suki': 0.04507080610021787, 'takaburi': 0.012254901960784314, 'ikari': 0.04874727668845316, 'aware': 0.08455882352941177, 'haji': 0.003812636165577342}
{'iya': 0.3610529464552797, 'odoroki': 0.026921926413401135, 'yasu': 0.09871373018247083, 'yorokobi': 0.14522883637451392, 'kowa': 0.18037690696978761, 'suki': 0.047711636254860905, 'takaburi': 0.014208794495961711, 'ikari': 0.04576727490278193, 'aware': 0.07627879150463655, 'haji': 0.0037391564463057133}
{'iya': 0.28242111496489297, 'odoroki': 0.079730987226123, 'yasu': 0.05710176803992894, 'yorokobi': 0.0826072244310972, 'kowa': 0.2494712799255562, 'suki': 0.043693426952034514, 'takaburi': 0.00854411640301159, 'ikari': 0.031554014042805174, 'aware': 0.1623805092631757, 'haji': 0.0024955587513746723}


    index       date         厭         驚         安         喜         怖  \
0       0 2020-03-01  0.336608  0.026444  0.097787  0.150216  0.200900   
1       0 2020-03-02  0.364211  0.025474  0.094059  0.142781  0.177162   
2       0 2020-03-03  0.351363  0.024254  0.096265  0.156471  0.183730   
3       0 2020-03-04  0.356972  0.022389  0.099102  0.143539  0.187180   
4       0 2020-03-05  0.349498  0.023582  0.098198  0.143669  0.193252   
5       0 2020-03-06  0.371403  0.024375  0.091846  0.145998  0.178791   
6       0 2020-03-07  0.344597  0.023570  0.102655  0.142837  0.194093   
7       0 2020-03-08  0.329245  0.023653  0.099431  0.140166  0.215214   
8       0 2020-03-09  0.344022  0.042040  0.097210  0.129782  0.208055   
9       0 2020-03-10  0.358807  0.030097  0.101387  0.139667  0.185298   
10      0 2020-03-11  0.357116  0.027234  0.093530  0.149969  0.171288   
11      0 2020-03-12  0.362080  0.034496  0.086623  0.147694  0.170691   
12      0 2020-03-13  0.359665  0.0357

{'iya': 0.3562696094341664, 'odoroki': 0.026722925457102673, 'yasu': 0.090014064697609, 'yorokobi': 0.12831331818673591, 'kowa': 0.19917775613978145, 'suki': 0.04100400302931949, 'takaburi': 0.01179270799523964, 'ikari': 0.04392513253272747, 'aware': 0.09953478307908688, 'haji': 0.0032456994482310936}
処理時間 0:08:00.829570


### (参考) ML-Ask の出力する全ての感情をカウントするコード

In [85]:
# ML-Ask の出力する結果のうち emotion を用いた結果
from mlask import MLAsk
import datetime
import sys

'''
ファイルの形式は
status_id \t retweet_count \t favorite_coun \t text
'''
def emotion_estimation(filename, emotions, verb=False):
    import os
    if os.path.exists(filename) is False:
        return []
    
    emotion_analyzer = MLAsk()
    
    emotion_counter = dict()
    for emotion in emotions:
        emotion_counter[emotion] = 0
        
    emotional_tweet_counter = 0
    # 開始時刻をメモ
    # start_time = datetime.datetime.now()
    
    with open(filename, "r", encoding='utf-8') as f:
        for line in f.readlines():
            # 何故か分からないが，(id, retweet_count 等が含まれないテキストのみの行が存在する
            try:
                status_id, retweet_count, favorite_count, text = line.rstrip('\n').split('\t')
            except ValueError:
                continue
                # pass
                # print(line)
                # sys.exit(1)
                
            status_id = int(status_id)
            retweet_count = int(retweet_count)
            favorite_count = int(favorite_count)
            
            # URL を含むツイートを排除（超簡易版の実装）
            if "http" in text:
                continue
                
            emotion = emotion_analyzer.analyze(text)['emotion']
            if emotion is not None:
                emotional_tweet_counter += 1
                
                # デバッグ用の表示
                if verb == True:
                    print("--------------------")
                    print(text)
                    print(emotion)
                    
                for k in emotion:
                    # print(k)
                    try:
                        emotion_counter[k] += 1
                    except:
                        emotion_counter[k] = 1
    
    for key in emotion_counter:
        emotion_counter[key] /= emotional_tweet_counter
        
    return emotion_counter

def set_period(from_year, from_month, from_day, until_year, until_month, until_day):
    data = list()
    # d = datetime.datetime(2020, 4, 1)
    d = datetime.datetime(from_year, from_month, from_day)
    while(True):
        if d > datetime.datetime(until_year, until_month, until_day): break
        data.append("{}{:02d}{:02d}".format(d.year, d.month, d.day))
        d += datetime.timedelta(days=1)
    return data

if __name__ == '__main__':
    # 開始時刻をメモ
    start_time = datetime.datetime.now()
    
    results = dict()

    emotions = ['iya', 'odoroki', 'yasu', 'yorokobi', 'kowa', 'suki', 'takaburi', 'ikari', 'aware', 'haji']
    j_emotions = {'iya':'厭', 'odoroki':'驚', 'yasu':'安', 'yorokobi':'喜', 'kowa':'怖', 'suki':'好', 'takaburi':'昂', 'ikari':'怒', 'aware':'哀', 'haji':'恥'}

    dates = set_period(2020, 3, 1, 2020, 7, 2)
    for d in dates:
        if d == "20200523": continue # データ取り忘れのため
        filename = "/home/muto/Dropbox/ExpSSE3/Tweets/{}_MERS.txt".format(d)
        # print(filename)
        emotion_counter = emotion_estimation(filename, emotions)
        print(emotion_counter, file=sys.stderr)
        
        # データを取れていない日に対処するための措置
        if emotion_counter == []: continue
            
        results[d] = emotion_counter
    
    # 終了時刻をメモ
    end_time = datetime.datetime.now()
    
    print("処理時間 {}".format(end_time - start_time))
    
    import pandas as pd
    df = pd.DataFrame(columns=['date']+emotions)
    
    for d in dates:
        dt = datetime.datetime(int(d[0:4]), int(d[4:6]), int(d[6:8]))
        try:
            df = df.append(pd.DataFrame([[dt, results[d]['iya'], results[d]['odoroki'], results[d]['yasu'],
                                          results[d]['yorokobi'], results[d]['kowa'],results[d]['suki'],
                                          results[d]['takaburi'], results[d]['ikari'], results[d]['aware'],
                                          results[d]['haji']]],
                                         columns=['date']+emotions))
        except:
            pass # データを取れていない日に対処するための措置
    
    df = df.rename(columns=j_emotions)
    
    print(df)
    
    import plotly_express as px
    df_melt = df.melt(id_vars='date', value_vars=j_emotions.values()) # value_vars=emotions)
    fig = px.line(df_melt, x="date", y="value", color='variable',
                  title="Emotions ....")
    fig.show()
    
    # matplotlib による可視化
    '''
    import matplotlib.pyplot as plt
    from matplotlib.font_manager import FontProperties
    fp = FontProperties(fname=r'/usr/share/fonts/truetype/fonts-japanese-gothic.ttf', size=16)

    for emotion in emotions:
        x = []
        for d in dates:
            try:
                x.append(results[d][emotion])
            except:
                # 当該の感情が全く含まれない場合はゼロとする (「恥」が少ない傾向にあるため)
                x.append(0)
        plt.plot(dates, x, label=j_emotions[emotion])
    plt.legend(loc='best', prop=fp)
    plt.savefig('result.png')
    plt.show()
    '''

{'iya': 0.41492975851620606, 'odoroki': 0.03828849508768708, 'yasu': 0.125516481498485, 'yorokobi': 0.21320356257460288, 'kowa': 0.26168395923239374, 'suki': 0.0750160683132862, 'takaburi': 0.04499127720135892, 'ikari': 0.05729501423193462, 'aware': 0.09236984666238178, 'haji': 0.007620971444311817}
{'iya': 0.4454440188830498, 'odoroki': 0.038300525518838516, 'yasu': 0.11828627416050592, 'yorokobi': 0.20566491493720496, 'kowa': 0.23300970873786409, 'suki': 0.07303821145452925, 'takaburi': 0.05041418010154093, 'ikari': 0.06965351385053888, 'aware': 0.08613164692259731, 'haji': 0.009619666874498976}
{'iya': 0.42487658295771624, 'odoroki': 0.036595836016312513, 'yasu': 0.12191457394290621, 'yorokobi': 0.2106675252200043, 'kowa': 0.24200472204335693, 'suki': 0.0721184803605924, 'takaburi': 0.04625456106460614, 'ikari': 0.0635329469843314, 'aware': 0.08682120626743936, 'haji': 0.010409959218716463}
{'iya': 0.43675417661097854, 'odoroki': 0.031026252983293555, 'yasu': 0.12615069894306172, 'y

{'iya': 0.4389769667962908, 'odoroki': 0.040532455877953934, 'yasu': 0.12458869279090637, 'yorokobi': 0.19682919533353274, 'kowa': 0.23706251869578224, 'suki': 0.07418486389470536, 'takaburi': 0.048758600059826504, 'ikari': 0.05279688902183667, 'aware': 0.09303021238408615, 'haji': 0.010469638049655998}
{'iya': 0.3960324845613738, 'odoroki': 0.11077742999746214, 'yasu': 0.08028085610354455, 'yorokobi': 0.16356484222992979, 'kowa': 0.33956518061077745, 'suki': 0.09512731579392607, 'takaburi': 0.038152440571863636, 'ikari': 0.04233990356145842, 'aware': 0.20793503087725235, 'haji': 0.006513831317147449}
{'iya': 0.4533160229362761, 'odoroki': 0.041761332900573406, 'yasu': 0.11457319052255761, 'yorokobi': 0.18532943849399547, 'kowa': 0.2664719246997728, 'suki': 0.07097262793465325, 'takaburi': 0.043708752569512066, 'ikari': 0.05322947095098994, 'aware': 0.12679865844422805, 'haji': 0.009304338418262469}
{'iya': 0.4526606772633034, 'odoroki': 0.0396222068647777, 'yasu': 0.1179451739230592, 

{'iya': 0.44579397210976157, 'odoroki': 0.029464687359424202, 'yasu': 0.12528115159694106, 'yorokobi': 0.18690958164642374, 'kowa': 0.22784525416104365, 'suki': 0.07624831309041835, 'takaburi': 0.04768331084120558, 'ikari': 0.07309941520467836, 'aware': 0.11178587494376968, 'haji': 0.010571300044984255}
{'iya': 0.4518743667679838, 'odoroki': 0.029381965552178316, 'yasu': 0.12386018237082067, 'yorokobi': 0.20542046605876393, 'kowa': 0.22416413373860183, 'suki': 0.07446808510638298, 'takaburi': 0.04685916919959473, 'ikari': 0.06357649442755826, 'aware': 0.10106382978723404, 'haji': 0.009371833839918946}
{'iya': 0.4407176287051482, 'odoroki': 0.04030161206448258, 'yasu': 0.12402496099843993, 'yorokobi': 0.20358814352574103, 'kowa': 0.2204888195527821, 'suki': 0.07878315132605304, 'takaburi': 0.0514820592823713, 'ikari': 0.06032241289651586, 'aware': 0.1032241289651586, 'haji': 0.013000520020800831}
{'iya': 0.46506087877183694, 'odoroki': 0.03493912122816305, 'yasu': 0.12519851773425092, '

{'iya': 0.4105314480741102, 'odoroki': 0.03217942467089225, 'yasu': 0.15114578254509994, 'yorokobi': 0.26474890297415893, 'kowa': 0.19502681618722575, 'suki': 0.14431984397854705, 'takaburi': 0.03803022915650902, 'ikari': 0.05802047781569966, 'aware': 0.07411019015114578, 'haji': 0.01121404193076548}
{'iya': 0.445822102425876, 'odoroki': 0.03234501347708895, 'yasu': 0.14932614555256066, 'yorokobi': 0.21940700808625338, 'kowa': 0.18328840970350405, 'suki': 0.11536388140161725, 'takaburi': 0.039353099730458224, 'ikari': 0.07061994609164421, 'aware': 0.07439353099730459, 'haji': 0.010242587601078167}
{'iya': 0.4369846878680801, 'odoroki': 0.035924617196702, 'yasu': 0.14664310954063603, 'yorokobi': 0.19905771495877503, 'kowa': 0.2226148409893993, 'suki': 0.09481743227326266, 'takaburi': 0.04063604240282685, 'ikari': 0.05594817432273263, 'aware': 0.07597173144876325, 'haji': 0.011189634864546525}
{'iya': 0.419281232173417, 'odoroki': 0.034227039361095266, 'yasu': 0.13006274957216202, 'yorok

{'iya': 0.45245245245245247, 'odoroki': 0.044044044044044044, 'yasu': 0.14414414414414414, 'yorokobi': 0.2062062062062062, 'kowa': 0.2122122122122122, 'suki': 0.08708708708708708, 'takaburi': 0.05005005005005005, 'ikari': 0.04504504504504504, 'aware': 0.0960960960960961, 'haji': 0.012012012012012012}
{'iya': 0.4631294964028777, 'odoroki': 0.03776978417266187, 'yasu': 0.12589928057553956, 'yorokobi': 0.18615107913669066, 'kowa': 0.2356115107913669, 'suki': 0.06924460431654676, 'takaburi': 0.02697841726618705, 'ikari': 0.0368705035971223, 'aware': 0.09082733812949641, 'haji': 0.008992805755395683}
{'iya': 0.4368279569892473, 'odoroki': 0.024193548387096774, 'yasu': 0.12231182795698925, 'yorokobi': 0.1975806451612903, 'kowa': 0.25268817204301075, 'suki': 0.0793010752688172, 'takaburi': 0.0456989247311828, 'ikari': 0.05913978494623656, 'aware': 0.0793010752688172, 'haji': 0.013440860215053764}
{'iya': 0.44333333333333336, 'odoroki': 0.03666666666666667, 'yasu': 0.14222222222222222, 'yoroko